In [1]:
import time
import core.utils as utils
from core.yolov4 import YOLOv4
from PIL import Image
from core.config import cfg

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
STRIDES = np.array(cfg.YOLO.STRIDES)
ANCHORS = utils.get_anchors(cfg.YOLO.ANCHORS, False)
NUM_CLASS = len(utils.read_class_names(cfg.YOLO.CLASSES))
XYSCALE = cfg.YOLO.XYSCALE

input_size = 416
image_path = "/home/lordgrim/Work/VisioLab/smaller-data/Images/group_2028.jpg"

original_image = cv2.imread(image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
original_image_size = original_image.shape[:2]

image_data = utils.image_preprocess(np.copy(original_image), [input_size, input_size])
image_data = image_data[np.newaxis, ...].astype(np.float32)

In [6]:
model_path = "models/final_tf_model5000"
model = load_model(model_path)
model.summary()

ion_87[0][0]     
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, 52, 52, 256)  294912      tf_op_layer_LeakyRelu_15[0][0]   
__________________________________________________________________________________________________
batch_normalization_88 (BatchNo (None, 52, 52, 256)  1024        conv2d_88[0][0]                  
__________________________________________________________________________________________________
tf_op_layer_LeakyRelu_16 (Tenso (None, 52, 52, 256)  0           batch_normalization_88[0][0]     
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 52, 52, 128)  32768       tf_op_layer_LeakyRelu_16[0][0]   
__________________________________________________________________________________________________
batch_normalization_89 (BatchNo (None, 52, 52, 128)  512         conv2d_89[0][0]           

In [13]:
pred_bbox = model.predict(image_data)
pred_bbox = utils.postprocess_bbbox(pred_bbox, ANCHORS, STRIDES, XYSCALE)
bboxes = utils.postprocess_boxes(pred_bbox, original_image_size, input_size, 0.25)
bboxes = utils.nms(bboxes, 0.213, method='nms')

image = utils.draw_bbox(original_image, bboxes)
image = Image.fromarray(image)
image.show()

In [12]:
print(bboxes)

[array([167.3993988 , 205.16627502, 322.99319458, 363.79983521,
         0.99763471,  16.        ]), array([250.07762146,  96.35608673, 439.00515747, 187.09532166,
         0.99838614,   9.        ]), array([486.77212524,  69.66133118, 674.65148926, 289.2583313 ,
         0.99960577,  12.        ]), array([274.43841553, 187.59390259, 606.94641113, 540.71954346,
         0.99970692,  20.        ])]


In [14]:
import coremltools as ct

# convert to Core ML and check predictions
mlmodel = ct.convert(model)


Translating MIL ==> MLModel Ops: 100%|██████████| 1937/1937 [00:06<00:00, 303.46 ops/s]


In [15]:
mlmodel.save("test1.mlmodel")